<a href="https://colab.research.google.com/github/anisha98/DiffusionScore/blob/main/Heat_values_generation_for_new_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!unzip /content/journalwithsnip.zip

Archive:  /content/journalwithsnip.zip
  inflating: journalwithsnip.csv     


In [ ]:
import math
import queue
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

In [ ]:
## Function to find all the adjacent nodes to each node in the network
def adjacency_matrix(visited,stack,stack_alpha): 
    g = data
    rows = g.shape[0]
    cols = g.shape[1]
    for x in range(0, rows):
        l = []
        for y in range(0, cols):
            if(g[x,y] == 1):
                l.append(y)
        adj.append(l)
    #print(adj)

In [ ]:
# Function to define the parent nodes of the current node
def parent_create(d, parent):
  for j in d:
      parent[j] = []
      for i in range(len(adj)):
          if j in adj[i]:
            parent[j].append(i)
#parent   

In [ ]:
## Depth First Search function is used to find the distance of each node from each other in the network
def DFS(v,visited,stack,stack_alpha):
    visited[v] = 1
    stack.append(v)
    for i in adj[v]:
        if visited[i] == 0:
            stack_alpha[i] = stack_alpha[v] + 1.0 # if there is a link between two nodes then the distance is incremented
            DFS(i,visited,stack,stack_alpha)
    #print("Stack_alpha",stack_alpha)
    return stack_alpha

In [ ]:
## Depth First Search function is used to combine all the values of closeness centrality and degree centrality and value of
## stack_alpha which is the value of the parent node of the current node
def Depth(v,visited,stack,stack_alpha):
    visited[v] = 1
    stack.append(v)
    #print("Visited",v)
    #print("Stack",stack)
    for i in adj[v]:
        #print("Current",i)
        if visited[i] == 0:
            stack_alpha[i] = (stack_alpha[v]) - d[i,0]/N + c[i,0]/N
            Depth(i,visited,stack,stack_alpha)
    #print("Stack_alpha",stack_alpha)
    return stack_alpha

In [ ]:
def heat_layer(y):
    return sum(heat[y])

In [ ]:
df1 = pd.DataFrame(columns = ['University','Article_ID','Snip','Score','Heat_values','Layers','Hops','Direct_citation','Indirect_citation','Total_Nodes','UScore'])

In [ ]:
data_snip = pd.read_csv('/content/journalwithsnip.csv')
snip = data_snip['id'].tolist()
data_snip.head()
#snip

,Unnamed: 0,doc_type,fos,snip,id,meta,organization,publisher,references,source_and_target,title,venue,year
0,0,1,"['computer vision', 'image segmentation', 'nat...",8.08,1969616664,{2562555919: {'organization': ['national unive...,"['stony brook university', 'stony brook univer...",IEEE Computer Society,"[1528802670, 1584193343, 1601567445, 168784646...","[[2437478628, 2328078142], [2511548893, 232807...",BabyTalk: Understanding and Generating Simple ...,IEEE Transactions on Pattern Analysis and Mach...,2013
1,1,1,"['statistical classification', 'theoretical co...",4.69,2050386482,{2126544388: {'organization': ['dept. of elect...,"['institute of telecommun., vienna university ...",IEEE,"[105338397, 1483758791, 1634279436, 1751228448...","[[2725938612, 2510844205], [2510844205, 151358...",Distributed particle filtering in agent networ...,NaN,2013
2,2,1,"['support vector machine', 'cluster analysis',...",1.62,2077494984,{2745134596: {'organization': ['department of ...,"['stony brook university', 'university of hels...","Pergamon Press, Inc.","[1494289067, 1555648035, 1777939696, 179095494...","[[2075568737, 2007979335], [2293377724, 200797...",Live and learn from mistakes: A lightweight sy...,Information Processing and Management,2013
3,3,1,"['tropism', 'machine learning', 'phenomenologi...",1.19,2101273298,{2895331146: {'organization': ['virginia commo...,"['stony brook university', ', hebrew universi...",Frontiers Media SA,"[1971689790, 2029408292, 2073985768, 207570475...","[[1904086895, 2101273298], [2138674902, 210127...",Self-referential forces are sufficient to expl...,Frontiers in Neuroinformatics,2013
4,4,1,"['data mining', 'machine learning', 'artificia...",0.87,2123372882,{2803549185: {'organization': ['marmara univer...,['fzi research center for information technolo...,Taylor & Francis Group,"[44713956, 76044956, 1507455930, 1542433116, 1...","[[2213728383, 1500684598], [2736057823, 262690...",REAL-TIME COMPLEX EVENT RECOGNITION AND REASON...,Applied Artificial Intelligence,2012


In [ ]:
data_snip[['snip']].min()

snip    0.26
dtype: float64

In [ ]:
# Step number 1) Read the data from each university and for each adjacency matrix
directory = ''
names = ['Minnesota/']
#names = ['clemenson/','colorado/','missouri/','nevada/','new mexico/','north dakota/','oregon/','south carolina/','stony brook/','temple/','vermont/','west virginia/']
for i1 in names:
    final = os.listdir(directory+i1)
    for i,name in enumerate(final): # Iterate through all the files in directory + names
        num = int(name[:-4]) # Splitting the string to find the article id 
        if num in snip:
            univ = i1 # assign vairbale univerisity for the current university data
            if(directory+i1+name):
                print(i1+name)
                data = pd.read_csv(directory+i1+name) #Read data using Pandas
                idx = data.index[data['Unnamed: 0'] == num] ## Since the location of the source node in the adjacency matrix is not
                ## known we get the index of the source node from the matrix 
                idx.tolist() # convert the index value into a list for later use(this solves the type problem)
                data = data.drop('Unnamed: 0',axis = 1) ## preprocess the data to make it into a numpy array
                data = data.to_numpy()
                N = len(data[0]) ##Total no of nodes in the matrix 
                #print(N)
                #print("source heat",idx[0])
                # Initialise variables 
                visited = [0]*N # used for DFS
                stack = [] # used for DFS
                stack_alpha = [0]*N # used for DFS
                adj = [] # to store the adjacency matrix returned from the adjacency_matrix function 

                # function call to adjacency_matrix
                adjacency_matrix(visited,stack,stack_alpha)

                d1 = [i for i in range(0,N)]
                d1.pop(idx[0])
                parent = {}
                parent_create(d1,parent)
                #print(parent)
                alpha_matrix = [] # to store the distance of each node from one another
                for i in range(N):
                    initial_node = i
                    stack = []
                    stack_alpha = [0.0]*N
                    visited = [0]*N
                    stack_alpha[i] = 1.0
                    p = DFS(i,visited,stack,stack_alpha)
                    alpha_matrix.append(p)
                for i in range(N):
                    for j in range(N):
                        if(alpha_matrix[i][j] > 0.0):
                            alpha_matrix[i][j] -=1 ## decrementing the distance by one to make counting start from 0
                for i in range(N):
                    alpha_matrix[i][i] = 0.0
                #print("Alpha_matrix",alpha_matrix)
                layers = alpha_matrix

                ## for closeness centrality - calculated by summing all the values in the rows of the alpha_matrix
                c = np.zeros(shape=(N,1))
                c = np.zeros(shape=(N,1))
                for i in range(N):
                    sum_dist = 0
                    for j in range(N):
                        if(alpha_matrix[i][j] >0):
                            sum_dist += 1
                    c[i,0] = sum_dist

                #print(c)
                ## Defines the heat kernal
                H = np.zeros(shape=(N,N))
                row = H.shape[0]
                col = H.shape[1]
                d = np.zeros(shape=(N,1)) 
                din = np.zeros(shape=(N,1))
                dout = np.zeros(shape=(N,1))
                g = data
                #count = -10
                for y in range(0,row):
                    p = g[y] == np.array([0]*N)
                    equal = p.all()
                    for x in range(0,col):
                        dout[y,0] -= g[y,x] #outgoing nodes - should ideally distribute the heat to the next nodes
                        din[y,0] += g[x,y] #ingoing nodes - should ideally increase the heat of each node
                    d[y,0] = din[y,0] + dout[y,0]

                ## To avoid having null values in the closeness centrality array we replace the 0s with -1 else negate the values 
                ## for futher calculation purpose
                for i in range(N):
                    if c[i,0]== 0:
                        c[i,0] = 1
                #print(d)
                # to store the distance of each node from one another
                alpha_matrix = []
                for i in range(N):
                    initial_node = i
                    stack = []
                    stack_alpha = [0.0]*N
                    visited = [0]*N
                    stack_alpha[i] = 1.0
                    p = Depth(i,visited,stack,stack_alpha)
                    alpha_matrix.append(p)
                for i in range(N):
                    alpha_matrix[i][i] = -1.0 ## decrementing the distance by one to make counting start from 0

                #print(alpha_matrix)
                g = data
                for x in range(0,N):
                    for y in range(0,N):  
                        if g[y,x] == 1 and alpha_matrix[y][x] != 0:
                            H[x,y] = 1/alpha_matrix[y][x] #if there is a link between the two nodes x and y and alpha_matrix is not
                            # equal to zero then we inverse the value
                        else:
                            H[x,y] = -alpha_matrix[y][x]/N ## this is for all the other node combinations and hence we dont want
                            ## their influence to affect the calculated values much so we divide by N(No of total nodes in the network)

                initial_node = idx[0]
                arr = [0.0]*N
                arr[initial_node] = float(data_snip.loc[data_snip['id'] == num, 'snip'].iloc[0])
                ## to initiate the source node value to be equal to its snip or h-index value
                initial_heat = np.array(arr)
                ## using heat equation calculating the heat in the network where t = 1
                heat = np.dot((np.exp(H)),initial_heat)
                #print("initial array",initial_heat)
                #print("heat initially",heat)
                heat1 = [0.0]*N
                ## normalising the values in the range of 2-1
                for i in range(len(heat)):
                    heat1[i] = ( (heat[i] - heat.min()) / (heat.max() - heat.min()) ) * (2 - 1) + 1

                ## normalising the values in the range of 0-1
                #print(arr[initial_node])
                for i in range(len(heat1)):
                    heat1[i] = (heat1[i]/2.0) * arr[initial_node]

                for i in range(len(heat)):
                    heat[i] = heat1[i]

                #print("after normalisation",heat)
                t = np.array(layers)
                d1 = {} 
                for i in range(N):
                    d1[i] = []

                ## Code after from here onwards is to find the heat layer wise in the network
                def printLevels(graph, V, x): 
                  # array to store level of each node  
                  level = [[] for i in range(N)]
                  # create a queue  
                  stk = []
                  stk.append(x)
                  level[x].append(0)
                  marked = [False for i in range(N)]
                  while(len(stk) != 0):
                      x1 = stk.pop()
                      if marked[x1] == False:
                        for i in graph[x1]:
                            if(i == idx[0]):
                              pass
                            else:
                              stk.append(i)
                              #print(i)
                              o = parent.get(i)
                              for j in o:
                                if(level[j] != []):
                                  for k in level[j]:
                                      level[i].append(k + 1)
                                      level[i] = list(set(level[i]))
                      marked[x1] = True

                  # display all nodes and their levels  
                  #print(level)
                  for i in range(N):
                      #print(level[i])
                      d1[i] = level[i]
  
                # Driver Code  
                if __name__ == '__main__': 
                    
                    # adjacency graph for tree  
                    V = N
                    graph = adj
                    printLevels(graph, V, idx[0]) 
    
                ## contains data of which and all layers each node comes in
                final_dictionary =  d1 
                f_d = {}
                max_val = 0
                ## calculates the total number of hops in the network
                for key,values in final_dictionary.items():
                    f_d[key] = list(set(values))
                    max_i = max(f_d[key])
                    if(max_i > max_val):
                        max_val = max_i

                ## to calculate all the nodes present in each hops ex: hop1:[node1,node23], hop2:[node0,node12] ...
                d_layer = {} 
                for i in range(max_val+1):
                    d_layer[i] = []
                for i in range(max_val+1):
                    for key,values in f_d.items():
                        if i in values:
                            d_layer[i].append(key)

                ##length of first hop is the direct citation
                direct_citation = len(d_layer[1])
                o = d_layer.copy()
                o.pop(0)
                o.pop(1)
                indirect_citation = 0
                ## sum of length of all the other layers except the first one
                for i,values in o.items():
                    indirect_citation += len(values)
                if(indirect_citation > 1):
                    indirect_citation -=1
                tot = N
                avg_heat_layer = []
                d_layer.pop(0) ## this contains the source node we dont want the heat in the source to be there for the calculation
                ## of the heat in each layer

                ## to calculate heat in each layer
                for i in range(1,max_val+1):
                    avg_heat_layer.append(heat_layer(d_layer[i])/len(d_layer[i]))

                ## sum all the heats in each layer
                score = sum(avg_heat_layer)
                #print(num,len(d_layer[1]),d_layer[1],direct_citation,max_val)
                print("We are done entering the value")
                print("*"*100)
                ## append the values to a dataframe for saving purpose
                df1 = df1.append({'University': univ, 'Article_ID': num,'Snip':float(data_snip.loc[data_snip['id'] == num, 'snip'].iloc[0]),'Score':score,'doc_type':float(data_snip.loc[data_snip['id'] == num, 'doc_type'].iloc[0]),'Heat_values':heat,'Layers':d_layer,'Hops':float(max_val),'Direct_citation':float(direct_citation),'Indirect_citation':float(indirect_citation),'Total_Nodes':float(tot),'UScore':0.0}, ignore_index=True)
            else:
                continue

Minnesota/2004744336.csv
We are done entering the value
****************************************************************************************************
Minnesota/2031299022.csv
We are done entering the value
****************************************************************************************************
Minnesota/2042554576.csv
We are done entering the value
****************************************************************************************************
Minnesota/2058704078.csv
We are done entering the value
****************************************************************************************************
Minnesota/2141766442.csv
We are done entering the value
****************************************************************************************************
Minnesota/1799314127.csv
We are done entering the value
****************************************************************************************************
Minnesota/2027355980.csv
We are done entering the value
**

In [ ]:
len(df1)

32

In [ ]:
df1.to_csv("Minnesota.csv")